In [2]:
import $ivy.`org.flinkextended::flink-scala-api:1.17.1_1.1.0`

import org.apache.flinkx.api._
import org.apache.flinkx.api.serializers._

import $ivy.$
import org.apache.flinkx.api._
import org.apache.flinkx.api.serializers._

In [1]:
import $ivy.`org.apache.flink:flink-table-api-java:1.17.1`
import $ivy.`org.apache.flink:flink-table-api-java-bridge:1.17.1`
import $ivy.`org.apache.flink:flink-table-planner-loader:1.17.1`
import $ivy.`org.apache.flink:flink-table-runtime:1.17.1`
import $ivy.`org.apache.flink:flink-connector-base:1.17.1`
import $ivy.`org.apache.flink:flink-connector-files:1.17.1`
import $ivy.`org.apache.flink:flink-clients:1.17.1`
import $ivy.`org.apache.flink:flink-runtime-web:1.17.1`

import $ivy.`org.slf4j:slf4j-log4j12:1.7.15`

// Paimon deps
import $ivy.`org.apache.flink:flink-shaded-hadoop-2-uber:2.8.3-10.0`
import $ivy.`org.apache.paimon:paimon-flink-1.17:0.5.0-incubating`

import org.apache.flink.table.api._
import org.apache.flink.table.api.bridge.java.StreamTableEnvironment
import org.apache.flink.configuration.{Configuration, ConfigConstants}
import org.apache.flink.configuration.RestOptions.BIND_PORT

import scala.collection.JavaConverters._
import scala.jdk.OptionConverters._

import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import $ivy.$
import org.apache.flink.table.api._
import org.apache.flink.table.api.bridge.java.StreamTableEnvironment
import org.apache.flink.configuration.{Configuration, ConfigConstants}
import org.apache.flink.configuration.RestOptions.BIND_PORT
import scala.collection.JavaConverters._
import scala.jdk.OptionConverters._

In [3]:
val config = Configuration.fromMap(
  Map(
    ConfigConstants.LOCAL_START_WEBSERVER -> "true",
    BIND_PORT.key -> "8082",
    "execution.checkpointing.interval" -> "10 s"
  ).asJava
)

val env = StreamExecutionEnvironment.createLocalEnvironmentWithWebUI(config)
val tEnv = StreamTableEnvironment.create(env.getJavaEnv)
val settings = EnvironmentSettings.newInstance().inStreamingMode()
  .withConfiguration(env.getJavaEnv.getConfiguration.asInstanceOf[Configuration])
  .build()
val table = TableEnvironment.create(settings)

1 deprecation
1 deprecation (since 2.13.0)
2 deprecations in total; re-run with -deprecation for details
SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions 1.7.x or earlier.
SLF4J: Ignoring binding found at [jar:file:/Users/alexeyn/Library/Caches/Coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.15/slf4j-log4j12-1.7.15.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See https://www.slf4j.org/codes.html#ignoredBindings for an explanation.


config: Configuration = {execution.checkpointing.interval=10 s, rest.port=8081, rest.bind-port=8082, local.start-webserver=true}
env: StreamExecutionEnvironment = org.apache.flinkx.api.StreamExecutionEnvironment@279b26d3
tEnv: StreamTableEnvironment = org.apache.flink.table.api.bridge.java.internal.StreamTableEnvironmentImpl@4d08fb78
settings: EnvironmentSettings = org.apache.flink.table.api.EnvironmentSettings@6386105e
table: TableEnvironment = org.apache.flink.table.api.internal.TableEnvironmentImpl@5c23140f

In [ ]:
def runFlinkSql(sql: String) = 
  sql.split(";").map(_.trim).filterNot(_.isEmpty).foreach { s => 
    println(s"Executing: $s")
    table.executeSql(s).print
  }    

In [ ]:
runFlinkSql("""
  CREATE CATALOG my_catalog WITH (
    'type'='paimon',
    'warehouse'='file:/tmp/paimon'
  );
  USE CATALOG my_catalog
""")

In [ ]:
runFlinkSql("""
  CREATE TABLE word_count (
    word STRING PRIMARY KEY NOT ENFORCED,
    cnt BIGINT
  ) WITH (
    'write-only' = 'true' 
  );
  CREATE TEMPORARY TABLE word_table (
    word STRING
  ) WITH (
    'connector' = 'datagen',
    'fields.word.length' = '1'
  )
""")

In [ ]:
val result = table.executeSql("""
    INSERT INTO word_count
    SELECT word, COUNT(*) FROM word_table GROUP BY word
""")

In [ ]:
result.getJobClient.ifPresent(status => print(status.getJobStatus.get))

In [ ]:
result.getJobClient.ifPresent(_.cancel())